In [1]:
import os
import sys

sys.path.insert(0, '../')

In [2]:
import pandas as pd
import numpy as np

from src.models.model_common import select_features

/Users/sreejithkrishnanr/Tools/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
train_data = pd.read_hdf('../data/processed/train_86400000000000.hd5', "data", parse_dates=[1])

In [4]:
site_data = train_data.loc[train_data['SiteId'] == 3, :]
site_data.head()

,ForecastId,Timestamp,obs_id,SiteId,Value,Frequency,DayOfMonth,DayOfWeek,DayOfYear,DaysInMonth,...,ConsumptionBiWeeklyMean,ConsumptionMonthlyMean,ConsumptionDailyMeanPerSurfaceArea,ConsumptionDailyMeanPerTemperatureDiff,ConsumptionWeeklyMeanPerSurfaceArea,ConsumptionWeeklyMeanPerTemperatureDiff,ConsumptionBiWeeklyMeanPerSurfaceArea,ConsumptionBiWeeklyMeanPerTemperatureDiff,ConsumptionMonthlyMeanPerSurfaceArea,ConsumptionMonthlyMeanPerTemperatureDiff
900,41,2013-01-02,6330712,3,3.207035e+05,8.640000e+13,2,2,2,31,...,5.488345e+06,6.327091e+06,30.380316,1.909459e+05,308.437493,1.938587e+06,519.912168,3.267745e+06,599.366672,3.767131e+06
901,41,2013-01-03,853183,3,8.787556e+05,8.640000e+13,3,3,3,31,...,5.488345e+06,6.327091e+06,30.380316,7.046004e+05,308.437493,7.153487e+06,519.912168,1.205815e+07,599.366672,1.390091e+07
902,41,2013-01-04,5568758,3,1.037190e+06,8.640000e+13,4,4,4,31,...,5.488345e+06,6.327091e+06,83.244714,3.576392e+06,308.437493,1.325121e+07,519.912168,2.233667e+07,599.366672,2.575023e+07
903,41,2013-01-05,3772665,3,7.726745e+05,8.640000e+13,5,5,5,31,...,5.488345e+06,6.327091e+06,98.253220,3.620926e+06,308.437493,1.136685e+07,519.912168,1.916032e+07,599.366672,2.208846e+07
904,41,2013-01-06,5249102,3,1.274910e+06,8.640000e+13,6,6,6,31,...,5.488345e+06,6.327091e+06,73.195626,1.439335e+06,308.437493,6.065184e+06,519.912168,1.022367e+07,599.366672,1.178608e+07


In [6]:
x, y, groups, ts = select_features(site_data, 'D')

In [7]:
from xgboost import XGBRegressor

regressor = XGBRegressor()

In [46]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(regressor, {
    'learning_rate': [0.12],
    'max_depth': [4],
    'n_estimators': [100],
    'colsample_bytree': [0.5],
    'colsample_bylevel': [0.6],
    'gamma': [0],
    'reg_alpha': [1, 1.1, 1.2, 1.3],
    'reg_lambda': [1, 1.1, 1.2, 1.3]
}, n_jobs=8, cv=8, scoring='neg_mean_squared_error')

gs.fit(x, y)

GridSearchCV(cv=8, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'learning_rate': [0.12], 'max_depth': [4], 'n_estimators': [100], 'colsample_bytree': [0.5], 'colsample_bylevel': [0.6], 'gamma': [0], 'reg_alpha': [1, 1.1, 1.2, 1.3], 'reg_lambda': [1, 1.1, 1.2, 1.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [45]:
gs.best_params_

{'colsample_bylevel': 0.6,
 'colsample_bytree': 0.5,
 'gamma': 0,
 'learning_rate': 0.12,
 'max_depth': 4,
 'n_estimators': 100,
 'reg_alpha': 1,
 'reg_lambda': 1.1}

In [41]:
gs.best_score_

-3326222323546.479